## Usposabljanje modela CBoW

Ta zvezek je del [Učnega načrta za začetnike v AI](http://aka.ms/ai-beginners)

V tem primeru bomo preučili usposabljanje jezikovnega modela CBoW za pridobitev lastnega vektorskega prostora Word2Vec. Kot vir besedila bomo uporabili podatkovno zbirko AG News.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Najprej naložimo naš nabor podatkov in definiramo tokenizator ter besedišče. Nastavili bomo `vocab_size` na 5000, da nekoliko omejimo izračune.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## Model CBoW

CBoW se nauči napovedovati besedo na podlagi $2N$ sosednjih besed. Na primer, ko je $N=1$, dobimo naslednje pare iz stavka *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Tukaj je prva beseda sosednja beseda, ki jo uporabimo kot vhod, druga beseda pa je tista, ki jo napovedujemo.

Za izdelavo mreže, ki napoveduje naslednjo besedo, moramo sosednjo besedo podati kot vhod in dobiti številko besede kot izhod. Arhitektura mreže CBoW je naslednja:

* Vhodna beseda se prenese skozi plast vgrajevanja. Ta plast vgrajevanja bo naša vgrajevanje Word2Vec, zato jo bomo definirali ločeno kot spremenljivko `embedder`. V tem primeru bomo uporabili velikost vgrajevanja = 30, čeprav bi morda želeli eksperimentirati z večjimi dimenzijami (pravi Word2Vec ima 300).
* Vektor vgrajevanja se nato prenese skozi linearno plast, ki bo napovedala izhodno besedo. Zato ima `vocab_size` nevronov.

Za izhod, če uporabimo `CrossEntropyLoss` kot funkcijo izgube, moramo podati le številke besed kot pričakovane rezultate, brez enovektorskega kodiranja.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Priprava podatkov za učenje

Zdaj bomo napisali glavno funkcijo, ki bo iz besedila izračunala CBoW pare besed. Ta funkcija nam bo omogočila določiti velikost okna in bo vrnila niz parov - vhodno in izhodno besedo. Upoštevajte, da se ta funkcija lahko uporablja tako na besedah kot na vektorjih/tensorjih - kar nam bo omogočilo kodiranje besedila, preden ga posredujemo funkciji `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Pripravimo učni nabor podatkov. Pregledali bomo vse novice, poklicali `to_cbow`, da dobimo seznam parov besed, in te pare dodali v `X` in `Y`. Zaradi prihranka časa bomo upoštevali le prvih 10 tisoč novic - omejitev lahko enostavno odstranite, če imate več časa za čakanje in želite dobiti boljše vdelave :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Podatke bomo prav tako pretvorili v en nabor podatkov in ustvarili nalagalnik podatkov:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Prav tako bomo te podatke pretvorili v en nabor podatkov in ustvarili nalagalnik podatkov:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Zdaj začnimo s pravim treningom. Uporabili bomo optimizator `SGD` z dokaj visoko hitrostjo učenja. Lahko poskusite tudi z drugimi optimizatorji, kot je `Adam`. Za začetek bomo trenirali 10 epoh - to celico lahko ponovno zaženete, če želite še nižjo izgubo.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Preizkušanje Word2Vec

Za uporabo Word2Vec izvlecimo vektorje, ki ustrezajo vsem besedam v našem besedišču:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Poglejmo na primer, kako je beseda **Pariz** kodirana v vektor:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Zanimivo je uporabiti Word2Vec za iskanje sinonimov. Naslednja funkcija bo vrnila `n` najbližjih besed glede na podan vhod. Da jih najdemo, izračunamo normo $|w_i - v|$, kjer je $v$ vektor, ki ustreza naši vhodni besedi, in $w_i$ je kodiranje $i$-te besede v besedišču. Nato razvrstimo tabelo in vrnemo ustrezne indekse z uporabo `argsort`, ter vzamemo prvih `n` elementov seznama, ki kodirajo položaje najbližjih besed v besedišču.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Ključne točke

Z uporabo pametnih tehnik, kot je CBoW, lahko treniramo model Word2Vec. Prav tako lahko poskusite trenirati model skip-gram, ki je zasnovan za napovedovanje sosednje besede glede na osrednjo, in preverite, kako dobro deluje.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi izhajale iz uporabe tega prevoda.
